In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import tweepy
import json
import time
import yaml

In [2]:
twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv')
twitter_archive_enhanced.shape

(2356, 17)

In [3]:
twitter_archive_enhanced.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [4]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

with open('image_predictions.tsv', 'wb') as file:
    file.write(response.content)
    
image_predictions = pd.read_csv('image_predictions.tsv', sep='\t')
image_predictions.shape

(2075, 12)

In [5]:
image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [6]:
twitter_credentials = yaml.load(open('./twitter_credentials.yml'))

consumer_key = twitter_credentials['api']['api_consumer_key']
consumer_secret = twitter_credentials['api']['api_consumer_secret']
access_token = twitter_credentials['access']['access_token']
access_token_secret = twitter_credentials['access']['access_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, \
                 parser= tweepy.parsers.JSONParser(), \
                 wait_on_rate_limit=True, \
                 wait_on_rate_limit_notify=True)

/home/usama/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """Entry point for launching an IPython kernel.


In [7]:
missing_tweets = []
count = 0

start_time = time.asctime()
start_sec = time.time()

with open ('tweet_json.txt', 'a') as file:
    for tweet_id in twitter_archive_enhanced.tweet_id:
        print('Start: ' + str(count))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            file.write(json.dumps(tweet) + '\n')
            print('Tweet ID: ' + str(tweet_id))
        except Exception as tweet_error_msg:
            missing_tweets.append(tweet_id)
            print('Error!!\n Tweet ID: ' + str(tweet_id))
            print('Error Msg: ' + str(tweet_error_msg))
        print('End: ' + str(count))
        count+=1
        print('------------------------------')
        if count == 50:
            break
    print('The End')
    
end_time = time.asctime()
end_sec = time.time()

Start: 0
Tweet ID: 892420643555336193
End: 0
------------------------------
Start: 1
Tweet ID: 892177421306343426
End: 1
------------------------------
Start: 2
Tweet ID: 891815181378084864
End: 2
------------------------------
Start: 3
Tweet ID: 891689557279858688
End: 3
------------------------------
Start: 4
Tweet ID: 891327558926688256
End: 4
------------------------------
Start: 5
Tweet ID: 891087950875897856
End: 5
------------------------------
Start: 6
Tweet ID: 890971913173991426
End: 6
------------------------------
Start: 7
Tweet ID: 890729181411237888
End: 7
------------------------------
Start: 8
Tweet ID: 890609185150312448
End: 8
------------------------------
Start: 9
Tweet ID: 890240255349198849
End: 9
------------------------------
Start: 10
Tweet ID: 890006608113172480
End: 10
------------------------------
Start: 11
Tweet ID: 889880896479866881
End: 11
------------------------------
Start: 12
Tweet ID: 889665388333682689
End: 12
------------------------------
Start:

In [8]:
start_time , end_time

('Fri Mar 26 19:35:00 2021', 'Fri Mar 26 19:36:01 2021')

In [9]:
start_sec , end_sec

(1616769300.826169, 1616769361.0610442)

In [10]:
end_sec - start_sec

60.234875202178955

In [11]:
missing_tweets

[888202515573088257]

In [12]:
new_missing_tweets = []
count = 0

start_time = time.asctime()
start_sec = time.time()

with open ('tweet_json.txt', 'a') as file:
    for tweet_id in missing_tweets:
        print('Start: ' + str(count))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            file.write(json.dumps(tweet) + '\n')
            print('Tweet ID: ' + str(tweet_id))
        except Exception as tweet_error_msg:
            new_missing_tweets.append(tweet_id)
            print('Error!!\n Tweet ID: ' + str(tweet_id))
            print('Error Msg: ' + str(tweet_error_msg))
        print('End: ' + str(count))
        count+=1
        print('------------------------------')
        if count == 50:
            break
    print('The End')
    
end_time = time.asctime()
end_sec = time.time()

Start: 0
Error!!
 Tweet ID: 888202515573088257
Error Msg: [{'code': 144, 'message': 'No status found with that ID.'}]
End: 0
------------------------------
The End


In [13]:
start_time , end_time

('Fri Mar 26 19:36:01 2021', 'Fri Mar 26 19:36:01 2021')

In [14]:
start_sec , end_sec

(1616769361.135011, 1616769361.7913353)

In [15]:
end_sec - start_sec

0.6563243865966797

In [16]:
missing_tweets == new_missing_tweets

True